<a href="https://colab.research.google.com/github/J053n1l50n/Modulo-27-atividaed/blob/main/tratamentoipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import *

In [ ]:
Spark = SparkSession.builder.getOrCreate()

In [ ]:
df_video = Spark.read.csv('videos-stats.csv', header=True, inferSchema=True)
df_comentario= Spark.read.csv('comments.csv', header=True, inferSchema=True)

In [ ]:
campo=['Likes', 'Comments', 'Views' ]
for col in campo:
  nulo=df_video.filter(df_video[col].isNull())
  df_video=df_video.na.fill(0, subset=col)
  df_video.show()
  df_video.filter(df_video[col]==0).show()


+---+--------------------+-----------+------------+-------+--------+--------+-----------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|Comments|      Views|
+---+--------------------+-----------+------------+-------+--------+--------+-----------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0|   135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|  1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|  1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0|   922918.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513.0|  5155.0|  1855644.0|
|  5|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570.0|  1643.0|   943119.0|
|  6|Tech Monopolies: ...|jXf04bhcjbg|  2022-06-13|   tech|135047.0|  9367.0|  5937790.0|
|  7|I bought the STRA...|2TqOmtTAMRY|  2022-08-07|   tech|216935.0| 12605.0|  4782514.0|
|  8|15 Em

In [ ]:
df_video.printSchema()
df_video.show()
df_video.count()
df_comentario.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: double (nullable = false)
 |-- Comments: double (nullable = false)
 |-- Views: double (nullable = false)

+---+--------------------+-----------+------------+-------+--------+--------+-----------+
|_c0|               Title|   Video ID|Published At|Keyword|   Likes|Comments|      Views|
+---+--------------------+-----------+------------+-------+--------+--------+-----------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407.0|   672.0|   135612.0|
|  1|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779.0|  4306.0|  1758063.0|
|  2|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825.0|  3338.0|  1564007.0|
|  3|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566.0|  1426.0|   922918.0|
|  4|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|  

In [ ]:
calculo_geral=df_video.join(df_comentario, df_video['Video ID'] == df_comentario['Video ID'], 'inner')
calcular = calculo_geral.count()
print(calcular)
df_video=df_video.na.drop(subset=['Video ID'])
df_comentario=df_comentario.na.drop(subset=['Video ID'])
calculo_refeito=df_video.join(df_comentario, df_video['Video ID']==df_comentario['Video ID'], 'inner')
calculo_refeito.show()
calcular_refaz=calculo_refeito.count()
print(calcular_refaz)


18645
+---+--------------------+-----------+------------+-------+-------+--------+---------+---+-----------+--------------------+------+---------+
|_c0|               Title|   Video ID|Published At|Keyword|  Likes|Comments|    Views|_c0|   Video ID|             Comment| Likes|Sentiment|
+---+--------------------+-----------+------------+-------+-------+--------+---------+---+-----------+--------------------+------+---------+
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|  0|wAZZ-UWGVHI|Let's not forget ...|  95.0|      1.0|
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|  1|wAZZ-UWGVHI|Here in NZ 50% of...|  19.0|      0.0|
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|  2|wAZZ-UWGVHI|I will forever ac...| 161.0|      2.0|
|  0|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407.0|   672.0| 135612.0|  3|wAZZ-UWGVHI|Whenever I go to ...|   8.0|      0.0|
|  0|Ap

In [ ]:
sem_duplicata= df_video.dropDuplicates(['Video ID'])

In [ ]:
df_video= df_video.withColumn('Likes', df_video['Likes'].cast('int')).\
          withColumn('Comments',df_video['Comments'].cast('int')).\
          withColumn('Views',df_video['Views'].cast('int'))
df_video.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)



In [ ]:
df_comentario=df_comentario.withColumn('Likes Comment', df_comentario['Likes'].cast('int')).\
                      withColumn('Sentiment', df_comentario['Sentiment'].cast('int'))
df_comentario=df_comentario.drop('Likes')
df_comentario.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Sentiment: integer (nullable = true)
 |-- Likes Comment: integer (nullable = true)



In [ ]:
df_video=df_video.withColumn('Interaction',df_video['Likes']+df_video['Comments']+df_video['Views'])
df_video.printSchema()
df_video.show()

root
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: integer (nullable = true)

+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+
|               Title|   Video ID|Published At|Keyword| Likes|Comments|   Views|Interaction|Year|
+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407|     672|  135612|     139691|2022|
|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779|    4306| 1758063|    1839148|2022|
|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825|    3338| 1564007|    1631170|2022|
|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   

In [ ]:
df_video=df_video.withColumn('Published At', df_video['Published At'].cast('date'))
df_video.printSchema()
df_video.show()

root
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: integer (nullable = true)

+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+
|               Title|   Video ID|Published At|Keyword| Likes|Comments|   Views|Interaction|Year|
+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407|     672|  135612|     139691|2022|
|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779|    4306| 1758063|    1839148|2022|
|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825|    3338| 1564007|    1631170|2022|
|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   

In [ ]:
df_video=df_video.withColumn('Year', year(df_video['Published At']))
df_video.printSchema()
df_video.show()

root
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: integer (nullable = true)

+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+
|               Title|   Video ID|Published At|Keyword| Likes|Comments|   Views|Interaction|Year|
+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407|     672|  135612|     139691|2022|
|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779|    4306| 1758063|    1839148|2022|
|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825|    3338| 1564007|    1631170|2022|
|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   

In [ ]:
df_join_video_comments=df_video.join(df_comentario,df_video['Video ID']==df_comentario['Video ID'], 'inner').drop(df_comentario['Video ID'])
df_join_video_comments.show()
df_join_video_comments.printSchema()

+--------------------+-----------+------------+-------+-----+--------+-------+-----------+----+---+--------------------+---------+-------------+
|               Title|   Video ID|Published At|Keyword|Likes|Comments|  Views|Interaction|Year|_c0|             Comment|Sentiment|Likes Comment|
+--------------------+-----------+------------+-------+-----+--------+-------+-----------+----+---+--------------------+---------+-------------+
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|  0|Let's not forget ...|        1|           95|
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|  1|Here in NZ 50% of...|        0|           19|
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|  2|I will forever ac...|        2|          161|
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|  3|Whenever I go to ...|        0|

In [ ]:
df_us_videos=Spark.read.csv('USvideos.csv', header=True, inferSchema=True)
df_us_videos.printSchema()
df_us_videos.show()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               t

In [ ]:
df_join_video_usvideos=df_video.join(df_us_videos,df_video['title']==df_us_videos['title'], 'inner')
df_join_video_usvideos.show()
df_join_video_usvideos.printSchema()

+--------------------+-----------+------------+----------------+------+--------+---------+-----------+----+-----------+-------------+--------------------+-----------------+-----------+--------------------+--------------------+--------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|               Title|   Video ID|Published At|         Keyword| Likes|Comments|    Views|Interaction|Year|   video_id|trending_date|               title|    channel_title|category_id|        publish_time|                tags|   views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+--------------------+-----------+------------+----------------+------+--------+---------+-----------+----+-----------+-------------+--------------------+-----------------+-----------+--------------------+--------------------+--------+------+--------+-------------+---

In [ ]:
luna=['Title','Video ID','Published At','Keyword','Likes','Comments','Views','Interaction','Year']
for col in luna:
  nulo = df_video.filter(df_video[col].isNull()).count()
  print(nulo)

0
0
0
0
0
0
0
0
0


In [ ]:
df_video=df_video.drop('_c0')
df_video.show()

+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+
|               Title|   Video ID|Published At|Keyword| Likes|Comments|   Views|Interaction|Year|
+--------------------+-----------+------------+-------+------+--------+--------+-----------+----+
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech|  3407|     672|  135612|     139691|2022|
|The most EXPENSIV...|b3x28s61q3c|  2022-08-24|   tech| 76779|    4306| 1758063|    1839148|2022|
|My New House Gami...|4mgePWWCAmA|  2022-08-23|   tech| 63825|    3338| 1564007|    1631170|2022|
|Petrol Vs Liquid ...|kXiYSI7H2b0|  2022-08-23|   tech| 71566|    1426|  922918|     995910|2022|
|Best Back to Scho...|ErMwWXQxHp0|  2022-08-08|   tech| 96513|    5155| 1855644|    1957312|2022|
|Brewmaster Answer...|18fwz9Itbvo|  2021-11-05|   tech| 33570|    1643|  943119|     978332|2021|
|Tech Monopolies: ...|jXf04bhcjbg|  2022-06-13|   tech|135047|    9367| 5937790|    6082204|2022|
|I bought the STRA..

In [ ]:
df_video.write.mode('overwrite').option('header','true').parquet('videos-tratados-parquet')

In [ ]:
df_join_video_comments=df_join_video_comments.drop('_c0')
df_join_video_comments.show()

+--------------------+-----------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+
|               Title|   Video ID|Published At|Keyword|Likes|Comments|  Views|Interaction|Year|             Comment|Sentiment|Likes Comment|
+--------------------+-----------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Let's not forget ...|        1|           95|
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Here in NZ 50% of...|        0|           19|
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|I will forever ac...|        2|          161|
|Apple Pay Is Kill...|wAZZ-UWGVHI|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Whenever I go to ...|        0|            8|
|Apple Pay Is

In [ ]:
df_join_video_comments.write.mode('overwrite').option('header', 'true').parquet('videos-comments-tratados-parquet')